In [1]:
import itertools
import numpy as np
import pandas as pd
import regex as re
import selenium.webdriver as webdriver
import time

from geopy.geocoders import Nominatim
from geopy import distance
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
def km_calc(retiro,entrega):
    geolocator = Nominatim(user_agent="sanico")
    coordenadas_retiro = geolocator.geocode(retiro)
    coordenadas_entrega = geolocator.geocode(entrega)
    lat_retiro = coordenadas_retiro.latitude
    lng_retiro = coordenadas_retiro.longitude
    lat_entrega = coordenadas_entrega.latitude
    lng_entrega = coordenadas_entrega.longitude

    coords_1 = (lat_retiro, lng_retiro)
    coords_2 = (lat_entrega, lng_entrega)
    km = distance.geodesic(coords_1, coords_2).km
    return km

In [41]:
coef_small = -79.95
coef_medium = -24.45
coef_large = 104.4
coef_express = 144.08333333
coef_coordinado = -144.08333333
coef_cantidad_de_paquetes = 57.13030303
coef_cantidad_de_asistentes = 610
coef_km = 48 # 48
intercept = 50 #75

In [31]:
def formula(inputs):
    km = km_calc(inputs["retiro"],inputs["entrega"])
    if inputs["tamano"]=="small":
        coef_tamano = coef_small
    elif inputs["tamano"]=="medium":
        coef_tamano = coef_medium
    else:
        coef_tamano = coef_large
    
    if inputs["opcion_de_entrega"]=="express":
        coef_opcion = coef_express
    else:
        coef_opcion = coef_coordinado
    
    precio = km*coef_km+inputs["cantidad_de_asistentes"]*coef_cantidad_de_asistentes+\
             inputs["cantidad_de_paquetes"]*coef_cantidad_de_paquetes + coef_tamano + coef_opcion + intercept
    
    print("Los km estimados son de: ", km)
    print("El precio estimado es de: ", precio)

In [30]:
def get_results(search):

    url = "https://www.envigo.uy"
    browser = webdriver.Chrome("./chromedriver")
    browser.get(url)
    timeout = 15

    ### Clickear en cruz para quitar pop-up inicial:
    time.sleep(0.5)
    browser.find_element_by_xpath("//div[@id='modal-content']/div/button").click()

    ### Ingresar dirección de retiro:
    WebDriverWait(browser, timeout).until(
        EC.presence_of_element_located((By.ID, 'estimate_pickup_address'))
    )
    retiro = browser.find_element_by_id("estimate_pickup_address")
    retiro.send_keys(search['retiro'])
    
    WebDriverWait(browser, timeout).until(
        EC.presence_of_element_located((By.XPATH, "//div[@class='pac-container pac-logo hdpi']/div[1]"))
    )
    WebDriverWait(browser, timeout).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='pac-container pac-logo hdpi']/div[1]"))
    )
    browser.find_element_by_xpath("//div[@class='pac-container pac-logo hdpi']/div[1]").click()

    ### Ingresar dirección de entrega:
    WebDriverWait(browser, timeout).until(
        EC.presence_of_element_located((By.ID, 'estimate_delivery_address'))
    )
    entrega = browser.find_element_by_id("estimate_delivery_address")
    entrega.send_keys(search['entrega'])
    WebDriverWait(browser, timeout).until(
        EC.presence_of_element_located((By.XPATH, "//div[@class='pac-container pac-logo hdpi']/div[1]"))
    )
    WebDriverWait(browser, timeout).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='pac-container pac-logo hdpi']/div[1]"))
    )
    browser.find_element_by_xpath("//div[@class='pac-container pac-logo hdpi']/div[1]").click()
    time.sleep(1)
    ### Latitudes y longitudes

    # lat_retiro = browser.find_element_by_id("estimate_pickup_lat").get_attribute('value')
    # lng_retiro = browser.find_element_by_id("estimate_pickup_lng").get_attribute('value')
    # lat_entrega = browser.find_element_by_id("estimate_pickup_lat").get_attribute('value')
    # lng_entrega = browser.find_element_by_id("estimate_pickup_lng").get_attribute('value')

    geolocator = Nominatim(user_agent="sanico")
    coordenadas_retiro = geolocator.geocode(search["retiro"])
    coordenadas_entrega = geolocator.geocode(search["entrega"])
    lat_retiro = coordenadas_retiro.latitude
    lng_retiro = coordenadas_retiro.longitude
    lat_entrega = coordenadas_entrega.latitude
    lng_entrega = coordenadas_entrega.longitude

    coords_1 = (lat_retiro, lng_retiro)
    coords_2 = (lat_entrega, lng_entrega)
    km = distance.geodesic(coords_1, coords_2).km

    ### Ingresar tamaño de paquete:
    if search["tamano"]=="small":
        WebDriverWait(browser, timeout).until(
            EC.presence_of_element_located((By.ID, 'estimate_package_size_small'))
        )
        WebDriverWait(browser, timeout).until(
        EC.element_to_be_clickable((By.ID, 'estimate_package_size_small'))
        )
        browser.find_element_by_xpath("//div[@id='package-size']/div/label[1]").click()
    elif search["tamano"]=="medium":
        WebDriverWait(browser, timeout).until(
            EC.presence_of_element_located((By.ID, 'estimate_package_size_medium'))
        )
        WebDriverWait(browser, timeout).until(
            EC.element_to_be_clickable((By.ID, 'estimate_package_size_medium'))
        )
        browser.find_element_by_xpath("//div[@id='package-size']/div/label[2]").click()
    elif search["tamano"]=="large":
        WebDriverWait(browser, timeout).until(
            EC.presence_of_element_located((By.ID, 'estimate_package_size_large'))
        )
        WebDriverWait(browser, timeout).until(
            EC.element_to_be_clickable((By.ID, 'estimate_package_size_large'))
        )
        browser.find_element_by_xpath("//div[@id='package-size']/div/label[3]").click()
    else:
        raise Exception('Valor no válido')

    ### Ingresar cantidad de paquetes:
    if int(search["cantidad_de_paquetes"])>=0 and int(search["cantidad_de_paquetes"])<=10:
        WebDriverWait(browser, timeout).until(
            EC.presence_of_element_located((By.ID, 'estimate_number_of_packages'))
        )
        cantidad_de_paquetes = browser.find_element_by_id("estimate_number_of_packages")
        cantidad_de_paquetes.send_keys(str(search["cantidad_de_paquetes"]))
    else:
        raise Exception('Valor no válido')

    ### Ingresar opción de entrega:
    if search["opcion_de_entrega"]=="express":
        browser.find_element_by_xpath("//div[@id='estimate_form_container']/div[2]/div[14]/div/label[1]").click()
    elif search["opcion_de_entrega"]=="coordinado":
        browser.find_element_by_xpath("//div[@id='estimate_form_container']/div[2]/div[14]/div/label[2]").click()
    else:
        raise Exception('Valor no válido')

    ### Ingresar cantidad de asistentes:
    if int(search["cantidad_de_asistentes"])>=0 and int(search["cantidad_de_asistentes"])<=5:
        cantidad_de_asistentes = browser.find_element_by_id("estimate_laborer_number")
        cantidad_de_asistentes.send_keys(search["cantidad_de_asistentes"])
    else:
        raise Exception('Valor no válido')

    time.sleep(1)
    precio_txt = browser.find_element_by_id("estimate_value_label").text
    precio = re.search("(?<=\$)\s(.*?)\s(?=IVA)",precio_txt)[1]
    resultado = [lat_retiro, lng_retiro, lat_entrega, lng_entrega, km, precio]
    print("El precio real es de: ", precio)
    browser.quit()

In [43]:
# search = {
#     "retiro" : "Rivera, Montevideo, Uruguay",
#     "entrega" : "Lieja, Montevideo, Uruguay",
#     "tamano" : "medium",
#     "opcion_de_entrega" : "express",
#     "cantidad_de_paquetes" : 2,
#     "cantidad_de_asistentes" : 2
#     }

# inputs = {
#     "retiro" : "Rivera, Montevideo, Uruguay",
#     "entrega" : "Lieja, Montevideo, Uruguay",
#     "tamano" : "medium",
#     "opcion_de_entrega" : "express",
#     "cantidad_de_paquetes" : 2,
#     "cantidad_de_asistentes" : 2,
#     }

search = {
    "retiro" : "Arocena, Montevideo, Uruguay",
    "entrega" : "Mantua, Montevideo, Uruguay",
    "tamano" : "medium",
    "opcion_de_entrega" : "express",
    "cantidad_de_paquetes" : 2,
    "cantidad_de_asistentes" : 2
    }

inputs = {
    "retiro" : "Arocena, Montevideo, Uruguay",
    "entrega" : "Mantua, Montevideo, Uruguay",
    "tamano" : "medium",
    "opcion_de_entrega" : "express",
    "cantidad_de_paquetes" : 2,
    "cantidad_de_asistentes" : 2,
    }

In [44]:
print(get_results(search))
print(formula(inputs))

El precio real es de:  1561
None
Los km estimados son de:  0.639573576392469
El precio estimado es de:  1534.5934710568386
None
